In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import sys; sys.path.insert(0, "../")
import sys; sys.path.insert(0, "../inverse_parametric_burgers/")

import numpy as np
from utils import *
from parametric_discovery_pinn import ParametricPINN, FinalParametricPINN
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR, DLrSR
from parametric_pde_diff import TrainSGTRidge, create_groups
from pytorch_robust_pca import R_pca

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

Populating the interactive namespace from numpy and matplotlib
You can use npar for np.array


In [2]:
data = pickle_load("../parametric_pde_data/parametric_ks.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]
Exact = data['u']

X, T = np.meshgrid(x, t)

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = min(20000, X_star.shape[0])
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

# Ground
L = 20
uu_x_true = 1 + 0.25*np.sin(x*2*np.pi/L)
u_xx_true = -1 + 0.25*np.exp(-(x-2)**2/5)
u_4x_true = -1 - 0.25*np.exp(-(x+2)**2/5)

eq_name = "ks"

Loaded from ../parametric_pde_data/parametric_ks.pkl


In [3]:
pinn = ParametricPINN(n_funcs=3, scale=True, lb=lb, ub=ub, eq_name=eq_name)
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)
pde_terms = ["u*u_x", "u_xx", "u_xxxx"]
func_terms = ["0.2442022*sin(0.3138633*x)+0.9938625", 
              "0.0922859381292682*exp(x*(1-0.24117917*x))-0.99314696", 
              "-0.12887639*exp(-0.18743528*(x**2)-0.708788816773944*x)-0.9929712"]
final_ks_pinn = FinalParametricPINN(model=model, pde_terms=pde_terms, 
                                    func_terms=func_terms, uncert=False, 
                                    scale=pinn.scale, lb=pinn.lb, ub=pinn.ub, 
                                    trainable_one=True
                                       )
del pinn

In [4]:
x, t = dimension_slicing(X_train)

In [5]:
def pcgrad_closure(return_list=False):
    global N, x, t, u_train
    losses = final_ks_pinn.loss(x, t, u_train)
    updated_grads = []
    
    for i in range(2):
        optimizer.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in final_ks_pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(final_ks_pinn.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    if not return_list: return losses[0]+losses[1]
    else: return losses

In [6]:
def finetuning_closure():
    global N, x, t, u_train
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # the solver network only consider the first N samples.
    mse_loss, pde_loss = final_ks_pinn.loss(x, t, u_train)
    loss = mse_loss + pde_loss
    if loss.requires_grad: loss.backward(retain_graph=False)
    return loss

#### GPU needed here...

In [7]:
optimizer = MADGRAD(final_ks_pinn.parameters(), lr=1e-2, momentum=0.9)
final_ks_pinn.train()

for i in range(200):
    optimizer.step(pcgrad_closure)
    if i%10==0:
        loss = pcgrad_closure(return_list=True)
        print(loss)
        
# f_opt = torch.optim.LBFGS(final_ks_pinn.parameters(), lr=1e-1, max_iter=300, max_eval=int(1.25*300), history_size=300, line_search_fn='strong_wolfe')
# for i in range(200):
#     f_opt.step(finetuning_closure)
#     if i%10==0:
#         loss = finetuning_closure()
#         print(loss.item())

[tensor(1.3560, grad_fn=<MseLossBackward>), tensor(2.8107e-09, grad_fn=<MseLossBackward>)]
[tensor(1.3500, grad_fn=<MseLossBackward>), tensor(5.0530e-08, grad_fn=<MseLossBackward>)]
[tensor(1.3502, grad_fn=<MseLossBackward>), tensor(1.8352e-08, grad_fn=<MseLossBackward>)]
[tensor(1.3495, grad_fn=<MseLossBackward>), tensor(8.9463e-08, grad_fn=<MseLossBackward>)]
[tensor(1.3492, grad_fn=<MseLossBackward>), tensor(9.6813e-08, grad_fn=<MseLossBackward>)]
[tensor(1.3485, grad_fn=<MseLossBackward>), tensor(2.9744e-07, grad_fn=<MseLossBackward>)]
[tensor(1.3478, grad_fn=<MseLossBackward>), tensor(3.6682e-07, grad_fn=<MseLossBackward>)]
[tensor(1.3467, grad_fn=<MseLossBackward>), tensor(2.8913e-07, grad_fn=<MseLossBackward>)]
[tensor(1.3431, grad_fn=<MseLossBackward>), tensor(7.3031e-07, grad_fn=<MseLossBackward>)]
[tensor(1.3460, grad_fn=<MseLossBackward>), tensor(6.8098e-07, grad_fn=<MseLossBackward>)]
[tensor(1.3534, grad_fn=<MseLossBackward>), tensor(1.3480e-09, grad_fn=<MseLossBackward>)]

In [8]:
save(final_ks_pinn, "./saved_path_inverse_parametric_ks/tmp.pth")